# Week 3 HW 오태환

In [2]:
import numpy as np
import pandas as pd

## 1. Convolution & Padding

In [4]:
img = np.array([2,0,2,1,3,2,0,2,0,2,2,2,1,0,1,3,1,1,0,0,1,1,1,0,0,1,3,4,1,0,0,1,0,0,5,2]).reshape(6,6)

In [6]:
filter = np.array([1,0,0,0,1,0,0,0,1]).reshape(3,3)

1-1 Zero Padding

In [7]:
pad = np.zeros(64).reshape(8,8)

In [8]:
for i in range(len(img)):
    for j in range(len(img)):
        pad[i+1][j+1] = img[i][j]

1-2 convolution

In [9]:
conv = np.zeros(36).reshape(6,6)
for i in range(len(img)):
    for j in range(len(img)):
        conved = pad[i:i+3, j:j+3] * filter
        conv[i][j] = conved.sum()

In [10]:
conv

array([[ 4.,  0.,  4.,  3.,  5.,  2.],
       [ 0.,  5.,  3.,  5.,  4.,  5.],
       [ 1.,  1.,  4.,  4.,  3.,  3.],
       [ 1.,  4.,  5.,  3.,  4.,  1.],
       [ 1.,  1.,  3., 10.,  4.,  1.],
       [ 0.,  1.,  1.,  3.,  9.,  3.]])

1-3 Max Pooling

In [11]:
pooled = np.zeros(9).reshape(3,3)

In [12]:
for i in range(0, len(conv),  2):
    for j in range(0, len(conv), 2):
        pool = conv[i:i+1, j:j+1]
        poolindx = int(i/2)
        poolindy = int(j/2)
        pooled[poolindx][poolindy] = pool.max()
        

In [13]:
pooled

array([[4., 4., 5.],
       [1., 4., 3.],
       [1., 3., 4.]])

## 2. CNN

In [14]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [16]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [17]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                         train = True,
                         transform = transforms.ToTensor(),
                         download = True)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [18]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [19]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
    #2-1 layer 1의 kernel size를 4로 늘리고, layer3의 stride를 2로 늘리기
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=4, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride=2))
        # conv_out = [1, 32, 27, 27]
        # pool_out = [1, 32, 13, 13]
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # conv_out = [?, 64, 13, 13]
        # pool_out = [?, 64, 6, 6]
        
    # 2-2 layer3의 output size를 구하기
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        # conv_out = [?, 128, 3, 3]
        # pool_out = [?, 128, 2, 2]
        
        self.fc1 = torch.nn.Linear(2 * 2 * 128, 625, bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p = 1 - self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

2-3 & 2-4 Cost & Accuracy 코딩으로 확인

In [20]:
model = CNN().to(device)

In [22]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.266454667
[Epoch:    2] cost = 0.0664872751
[Epoch:    3] cost = 0.0471392572
[Epoch:    4] cost = 0.0353944078
[Epoch:    5] cost = 0.0296183415
[Epoch:    6] cost = 0.0252030678
[Epoch:    7] cost = 0.0197279081
[Epoch:    8] cost = 0.0183607191
[Epoch:    9] cost = 0.0170458034
[Epoch:   10] cost = 0.0142284231
[Epoch:   11] cost = 0.011833475
[Epoch:   12] cost = 0.012941434
[Epoch:   13] cost = 0.0108358348
[Epoch:   14] cost = 0.00986428559
[Epoch:   15] cost = 0.00925068092


In [24]:
# Test model and check accuracy
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9825999736785889


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
